In [ ]:
import requests, json, csv

In [ ]:
#Словарь с данными для API
api = {
    'gateway': 'https://manifesto-project.wzb.eu/tools/',
    'key': '?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0'
}

In [ ]:
"""
    Обертка для обращений к API
    Аргументы:
        string : api_method – Название метода (https://manifestoproject.wzb.eu/information/documents/api)
        *string : params – Параметры вызова
"""
def api_request(api_method, *params):
    parameters = ''
    if params:
        for i in params:
            parameters += ('&' + i)
    request_url = api['gateway'] + api_method + '.json' + api['key'] + parameters
    print(request_url)
    return json.loads(requests.get(request_url).text)

"""
    Пишет CSV в папку datasheets
    Аргументы:
        list : data – Список данных для записи
        string : name – Имя файла с расширением
"""
def list_to_csv(data, name):
    path = 'datasheets/' + name
    with open(path, "w", newline='') as csv_file:
            writer = csv.writer(csv_file, delimiter=',')
            for line in data:
                writer.writerow(line)
                
def dict_to_csv(data, name):
    path = 'datasheets/' + name
    with open(path,'w') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow((data[0].keys()))
        for item in data:
            writer.writerow((item.values()))

"""
    Читает из CSV данные из выбранных колонок
    Аргументы:
        string : name – Имя файла с расширением
        *string : headings – Названия колонок
        (Optional) bool : write_first_row – Записывать ли в список заголовки колонок
"""
def csv_to_list(name, *headings, write_first_row=True):
    with open('datasheets/' + name) as file_obj:
        reader = csv.DictReader(file_obj, delimiter=',')
        result = []
        if write_first_row:
            first_row = []
            for heading in headings:
                first_row.append(heading)
            result.append(first_row)
        for line in reader:
            parsed = []
            for heading in headings:
                parsed.append(line[heading])
            result.append(parsed)
        return result

In [ ]:
"""
    Здесь собираем ключи для дальнейшего парсинга метаданных
    Надо будет обернуть в функцию, но мне лень
"""

core_set_from_csv = csv_to_list('core.csv', 'id_perm', 'party', 'date', write_first_row=False)

list_to_parse_metadata = []
for sign in core_set_from_csv:
    item = {
        'id_perm': sign[0],
        'key': sign[1] + '_' + sign[2]
    }
    list_to_parse_metadata.append(item)

In [ ]:
"""
    Парсим все метаданные и сохраняем в список
"""

metadata_list = []

for sign in list_to_parse_metadata:
    responce = api_request('api_metadata', 'keys[]=' + sign['key'], 'version=2017-2')
    metadata_list.append(responce['items'][0])

In [ ]:
"""
    Выбираем из списка метаданных подходящие нам манифесты
"""

manifestos_to_parse = []

for sign in metadata_list:
    if sign['language'] == 'english' and sign['annotations'] == True:
        item = {
            'party_id': sign['party_id'],
            'date': sign['election_date'],
            'manifesto_id': sign['manifesto_id'],
            'md5sum_text': sign['md5sum_text'],
            'title': sign['title']
        }
        manifestos_to_parse.append(item)

In [ ]:
"""
    Собираем корпус по метаданным отфильтрованным ранее
"""

corpus = []

for sign in manifestos_to_parse:
    responce = api_request('api_texts_and_annotations', 'keys[]=' + sign['manifesto_id'], 'version=2017-2')
    for item in responce['items'][0]['items']:
        if item['cmp_code'] != 'NA':
            parsed_item = {
                'text': item['text'],
                'cmp_code': item['cmp_code'],
                'party_id': sign['party_id'],
                'date': sign['date']
            }
            corpus.append(parsed_item)

In [38]:
corpus

[{'text': "Our aim: To make Britain the world's foremost learning society by 2010.",
  'cmp_code': '411',
  'party_id': 51421,
  'date': '199705'},
 {'text': "The problem: This country's education has been underfunded and undermined by repeated shifts in policy.",
  'cmp_code': '506',
  'party_id': 51421,
  'date': '199705'},
 {'text': 'Standards are too low, especially in core skills such as reading and maths.',
  'cmp_code': '506',
  'party_id': 51421,
  'date': '199705'},
 {'text': 'Britain is too low in the world league tables.',
  'cmp_code': '506',
  'party_id': 51421,
  'date': '199705'},
 {'text': "Our commitment: Liberal Democrats will make education the next government's top priority.",
  'cmp_code': '506',
  'party_id': 51421,
  'date': '199705'},
 {'text': 'We will invest an additional £2 billion per year in education, funded by an extra 1p in the pound on the basic rate of income tax.',
  'cmp_code': '506',
  'party_id': 51421,
  'date': '199705'},
 {'text': '- Give childr